### Combine All model prediction in one file

In [226]:
import pandas as pd
import numpy as np

In [227]:
veg1 = ['香蕉','洋香瓜','蓮霧','鳳梨','南瓜',
        '大蒜','青蔥','小白菜','芽菜類','木瓜']

veg2 = ['香蕉','洋香瓜-網狀紅肉','蓮霧-紅蓮霧','鳳梨-金鑽鳳梨','南瓜-木瓜形',
        '大蒜-蒜仁','青蔥-日蔥','小白菜-土白菜','芽菜類-黃豆牙','木瓜-網室紅肉']

models = ['LSTM1','LSTM2','ARIMA','Prophet']
Target = ['Price','3_day_return','price','3DayReturn','平均價','3_day_return']

In [315]:
# change vegetable class and predicted target
idx = 0
pred = 1
#############################################

path1 = './LSTM1_csv/2019_台北一_'+veg1[idx]+'_'+Target[pred]+'.csv'
model1 = pd.read_csv(path1)
path2 = './LSTM2_csv/'+veg2[idx]+'_'+Target[pred]+'.csv'
model2 = pd.read_csv(path2)
path3 = './ARIMA_csv/'+veg1[idx]+'_'+Target[pred+2]+'.csv'
model3 = pd.read_csv(path3)
path4 = './Prophet_csv/'+ veg2[idx]+'_'+Target[pred+4]+'.csv'
model4 = pd.read_csv(path4)

In [316]:
print(path1)
print(path2)
print(path3)
print(path4)

./LSTM1_csv/2019_台北一_香蕉_3_day_return.csv
./LSTM2_csv/香蕉_3_day_return.csv
./ARIMA_csv/香蕉_3DayReturn.csv
./Prophet_csv/香蕉_3_day_return.csv


In [317]:
model1 = model1.rename(columns = {'Price':'Price_'+str(idx),'3_day_return':'3_day_return_'+str(idx),
                                  'Price_pred':'LSTM1_'+str(idx),'3_day_return_pred':'LSTM1_'+str(idx)})

In [318]:
model1

,date,3_day_return_0,LSTM1_0
0,2019-01-01,-2.941176,12.699489
1,2019-01-02,-5.395683,13.503502
2,2019-01-03,1.486989,-5.430964
3,2019-01-04,1.893939,3.563771
4,2019-01-05,11.787072,18.646378
...,...,...,...
293,2019-12-25,8.750000,7.685295
294,2019-12-26,4.294479,2.712766
295,2019-12-27,-5.949008,-11.327789
296,2019-12-28,-6.321839,-4.028879


In [319]:
for i in range(1, len(model2)):
    model2['DateTime'][len(model2)-i] = model2['DateTime'][len(model2)-i-1]

c:\users\user\anaconda3\envs\lstm-shap\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [320]:
model2 = model2.iloc[1:]
model2 = model2[['DateTime','Predict']]

In [321]:
model2 = model2.rename(columns={'Predict':'LSTM2_'+str(idx),'DateTime':'date'})

In [322]:
model2

,date,LSTM2_0
1,2019-01-01,0.150518
2,2019-01-02,-3.356138
3,2019-01-03,-3.961330
4,2019-01-04,3.267810
5,2019-01-05,3.992990
...,...,...
294,2019-12-25,10.670970
295,2019-12-26,7.223215
296,2019-12-27,-2.059632
297,2019-12-28,-6.407492


In [323]:
model3 = model3[['DateTime','predict']]
model3 = model3.rename(columns={'predict':'ARIMA_'+str(idx),'DateTime':'date'})

In [324]:
model3

,date,ARIMA_0
0,2019-01-01,-0.185544
1,2019-01-02,0.260455
2,2019-01-03,5.298045
3,2019-01-04,6.077908
4,2019-01-05,4.063551
...,...,...
294,2019-12-26,1.131963
295,2019-12-27,-4.016639
296,2019-12-28,-3.859867
297,2019-12-29,-2.220156


In [325]:
model4 = model4[['DateTime','predict']]
model4 = model4.rename(columns={'predict':'Prophet_'+str(idx),'DateTime':'date'})

In [326]:
model4

,date,Prophet_0
0,2019-01-01,4.131940
1,2019-01-02,3.167573
2,2019-01-03,0.806897
3,2019-01-04,-1.523029
4,2019-01-05,-0.970798
...,...,...
291,2019-12-26,1.023290
292,2019-12-27,-1.101259
293,2019-12-28,-0.348878
294,2019-12-29,2.472809


In [327]:
All_Predict = model1.set_index('date').join(model2.set_index('date')).join(model3.set_index('date')).join(model4.set_index('date'))

In [328]:
All_Predict = All_Predict.reset_index()

In [329]:
All_Predict

,date,3_day_return_0,LSTM1_0,LSTM2_0,ARIMA_0,Prophet_0
0,2019-01-01,-2.941176,12.699489,0.150518,-0.185544,4.131940
1,2019-01-02,-5.395683,13.503502,-3.356138,0.260455,3.167573
2,2019-01-03,1.486989,-5.430964,-3.961330,5.298045,0.806897
3,2019-01-04,1.893939,3.563771,3.267810,6.077908,-1.523029
4,2019-01-05,11.787072,18.646378,3.992990,4.063551,-0.970798
...,...,...,...,...,...,...
293,2019-12-25,8.750000,7.685295,10.670970,8.794684,3.197643
294,2019-12-26,4.294479,2.712766,7.223215,1.131963,1.023290
295,2019-12-27,-5.949008,-11.327789,-2.059632,-4.016639,-1.101259
296,2019-12-28,-6.321839,-4.028879,-6.407492,-3.859867,-0.348878


In [330]:
All_Predict.to_csv('./All_Predict/All_Predict_'+veg2[idx]+'_'+Target[pred]+'.csv', index=False)

### Put all kind of vegetable prediction into a file

In [331]:
idx = 0
all_predict = pd.read_csv('./All_Predict/All_Predict_'+veg2[idx]+'_'+Target[pred]+'.csv')
all_predict = all_predict.set_index('date')

In [332]:
for i in range(1,10):
    dataset = pd.read_csv('./All_Predict/All_Predict_'+veg2[i]+'_'+Target[pred]+'.csv')
    dataset = dataset.set_index('date')
    all_predict = all_predict.join(dataset, how='outer')

In [333]:
all_predict = all_predict.reset_index()
all_predict

,date,3_day_return_0,LSTM1_0,LSTM2_0,ARIMA_0,Prophet_0,3_day_return_1,LSTM1_1,LSTM2_1,ARIMA_1,...,3_day_return_8,LSTM1_8,LSTM2_8,ARIMA_8,Prophet_8,3_day_return_9,LSTM1_9,LSTM2_9,ARIMA_9,Prophet_9
0,2019-01-01,-2.941176,12.699489,0.150518,-0.185544,4.131940,-1.412429,22.030544,-7.083355,-1.382656,...,0.500000,1.126456,1.514915,0.261587,-0.303947,39.037433,19.976370,31.670609,13.044277,-0.725810
1,2019-01-02,-5.395683,13.503502,-3.356138,0.260455,3.167573,-8.536585,15.129044,2.318338,-1.000117,...,0.000000,1.046462,1.405913,0.261587,-1.396031,64.736842,48.814330,38.440910,14.215867,0.080626
2,2019-01-03,1.486989,-5.430964,-3.961330,5.298045,0.806897,-1.618123,24.693531,-6.001196,4.402780,...,0.000000,1.380775,0.525142,0.261587,-1.498028,40.465116,22.328590,37.545475,10.353469,-0.217475
3,2019-01-04,1.893939,3.563771,3.267810,6.077908,-1.523029,13.467049,59.267480,-13.069328,4.440639,...,0.000000,2.907618,-1.487068,0.261587,-1.084711,16.923077,-14.574517,7.440484,5.435430,-4.146135
4,2019-01-05,11.787072,18.646378,3.992990,4.063551,-0.970798,43.333333,55.448696,1.462522,6.021344,...,-0.497512,3.032016,-1.680679,0.261587,0.309924,-4.792332,-27.266829,-8.307306,1.109468,-2.983827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,2019-12-25,8.750000,7.685295,10.670970,8.794684,3.197643,14.326648,14.615886,-3.663722,4.070499,...,0.000000,2.783752,-0.484782,0.260721,-0.582079,11.290323,7.584568,9.203438,1.890727,-3.372864
294,2019-12-26,4.294479,2.712766,7.223215,1.131963,1.023290,28.358209,2.643274,7.981291,8.179342,...,-0.497512,2.334932,-0.901029,0.260721,-0.734777,-0.903614,-0.397143,6.815876,4.393802,-3.721430
295,2019-12-27,-5.949008,-11.327789,-2.059632,-4.016639,-1.101259,24.157303,1.701463,5.524634,6.128176,...,5.970149,1.874161,0.241766,0.260721,-0.394503,-0.895522,-49.157223,-0.388340,-2.410720,-7.611605
296,2019-12-28,-6.321839,-4.028879,-6.407492,-3.859867,-0.348878,8.020050,-1.883283,5.495371,4.809966,...,0.500000,-1.191915,-0.317170,0.263073,0.903421,-4.927536,-45.962986,-8.040676,-7.423064,-6.351898


In [334]:
for i in range(len(all_predict)):
    for j in range(10):
        if j == 5: continue
        if pd.isnull(all_predict['Prophet_'+str(j)][i]):
            print(all_predict['date'][i],all_predict['Prophet_'+str(j)][i])
            all_predict['Prophet_'+str(j)][i] = all_predict['Prophet_'+str(j)][i-1]

2019-02-04 nan
2019-02-04 nan
2019-02-04 nan
2019-02-04 nan
2019-02-04 nan
2019-02-04 nan
2019-02-04 nan
2019-02-04 nan
2019-02-04 nan
2019-02-14 nan
2019-02-22 nan
2019-06-11 nan
2019-06-16 nan
2019-06-16 nan
2019-06-16 nan
2019-06-16 nan
2019-06-16 nan
2019-06-16 nan
2019-06-16 nan
2019-06-16 nan
2019-06-16 nan
2019-09-14 nan
2019-09-14 nan
2019-09-17 nan
2019-09-17 nan
2019-09-21 nan
2019-09-21 nan
2019-09-22 nan
2019-09-22 nan
2019-09-22 nan
2019-09-22 nan
2019-09-22 nan
2019-09-22 nan
2019-09-22 nan
2019-09-22 nan
2019-09-22 nan
2019-10-04 nan


c:\users\user\anaconda3\envs\lstm-shap\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [335]:
for i in range(len(all_predict)):
    for j in range(10):
        all_predict = all_predict[all_predict[Target[pred]+'_'+str(j)].notna()]

In [336]:
all_predict.to_csv('all_prediction_'+Target[pred]+'.csv', index=False)